In [3]:
import pandas as pd
import numpy as np

train_data = pd.read_csv('https://raw.githubusercontent.com/JadonLeung/STAT4012/main/data_train.csv')
test_data = pd.read_csv('https://raw.githubusercontent.com/JadonLeung/STAT4012/main/data_test.csv')

data = train_data.title
data.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

testdata = test_data.title
testdata.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

train_data['word_length'] = train_data['title'].apply(lambda x:len(x.split()))
train_data

datetime                                              title   
0    2021-10-17 18:00:00  Why the upcoming Bitcoin futures ETF may not b...  \
1    2021-10-18 07:43:00  Shiba Inu  SHIB  jumps    while Bitcoin  Ether...   
2    2021-10-18 11:00:00  Institutional Bitcoin exchange Bakkt goes publ...   
3    2021-10-18 14:35:00  US mortgage lender dumps Bitcoin plans just tw...   
4    2021-10-18 15:07:00  ProShares Bitcoin ETF confirmed for October   ...   
..                   ...                                                ...   
927  2023-01-03 17:00:00  Research  Only    K Bitcoin remain in Future O...   
928  2023-01-04 00:00:00  ATOM and SOL beat BTC returns over the past   ...   
929  2023-01-04 17:00:00   Research  Bitcoin UTXOs grow despite bear market   
930  2023-01-05 13:01:00  CryptoSlate Daily wMarket Update  Top    asset...   
931  2023-01-05 16:56:00  Pompliano and Shellenberger clash over validit...   

              datetime.1       ret  label  word_length  
0    2021-10-17 18:00:00 -0.000572      0           14  
1    2021-10-18 07:43:00 -0.011143     -1            9  
2    2021-10-18 11:00:00 -0.004227     -1           10  
3    2021-10-18 14:35:00 -0.000662      0           11  
4    2021-10-18 15:07:00  0.000761      0            7  
..                   ...       ...    ...          ...  
927  2023-01-03 17:00:00  0.000569      0           15  
928  2023-01-04 00:00:00 -0.000793      0           10  
929  2023-01-04 17:00:00  0.001123      0            7  
930  2023-01-05 13:01:00 -0.002714      0           12  
931  2023-01-05 16:56:00 -0.001342      0            8  

[932 rows x 6 columns]

In [18]:
for i,row in data.items(): 
    data.iloc[i]=data.iloc[i].lower()

for i,row in testdata.items(): 
    testdata.iloc[i]=testdata.iloc[i].lower()

def preprocessingData(preprocessData):
    list1= [i for i in preprocessData]
    new_Index=[str(i) for i in list1]
    preprocessData.columns=new_Index
    for i,row in preprocessData.items(): 
        preprocessData.iloc[i]=preprocessData.iloc[i].lower()
        
    return preprocessData
    
data = preprocessingData(data)
testdata = preprocessingData(testdata)

data
testdata
    


/tmp/ipykernel_7863/3295379664.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[i]=data.iloc[i].lower()
/tmp/ipykernel_7863/3295379664.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[i]=data.iloc[i].lower()
/tmp/ipykernel_7863/3295379664.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.iloc[i]=data.iloc[i].lower()
/tmp/ipykernel_7863/3295379664.py:2: SettingWithCopyWarning: 
A value is trying to be se

0      research  bitcoin super whales remain aggressi...
1      genesis owner digital currency group shuts dow...
2      mt  gox extends deadline for btc repayment reg...
3      bitcoin peaks to         on higher than expect...
4      poolin bitcoin mining hash rate share falls by...
                             ...                        
228    u s  government sold over  k btc for       m o...
229    more than     of bitcoin mining uses renewable...
230    senator warren says banks have  done really ba...
231    cryptoslate wmarket update  cardano leads top ...
232    bitfarms q  bitcoin production rose     to    ...
Name: title, Length: 233, dtype: object

In [5]:
from keras.preprocessing.text import Tokenizer
num_words = 10000 # this means 15000 unique words can be taken 
tokenizer=Tokenizer(num_words,lower=True)
df_total = pd.concat([data,testdata], axis = 0)
tokenizer.fit_on_texts(data)

In [6]:
len(tokenizer.word_index)

2594

In [7]:
len(testdata.max())

41

In [16]:
from keras_preprocessing.sequence import pad_sequences
import tensorflow as tf 
tf.config.run_functions_eagerly(True)

from tensorflow import keras

X_train_ =tokenizer.texts_to_sequences(data)
X_train_pad=pad_sequences(X_train_,maxlen=41,padding='post')
X_test_ = tokenizer.texts_to_sequences(testdata)
X_test_pad = pad_sequences(X_test_, maxlen = 41, padding = 'post')
x_test =  keras.utils.to_categorical(train_data.label, 3)
y_test = keras.utils.to_categorical(test_data.label, 3)

In [9]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, LSTM, Embedding,Bidirectional
import tensorflow
from tensorflow.compat.v1.keras.layers import CuDNNLSTM,CuDNNGRU
from tensorflow.keras.layers import Dropout

In [10]:
import tensorflow as tf
EMBEDDING_DIM = 100 # this means the embedding layer will create  a vector in 100 dimension
model = Sequential()
model.add(Embedding(input_dim = num_words,# the whole vocabulary size 
                          output_dim = EMBEDDING_DIM, # vector space dimension
                          input_length= X_train_pad.shape[1] # max_len of text sequence
                          ))
model.add(Dropout(0.2))
model.add(Bidirectional(CuDNNLSTM(100,return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(CuDNNLSTM(200,return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(CuDNNLSTM(100,return_sequences=False)))
model.add(Dense(3, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = tf.keras.optimizers.legacy.Adam(lr=1e-5),metrics = 'accuracy')

2023-04-17 14:52:07.700513: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-17 14:52:07.726665: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-17 14:52:07.726715: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-17 14:52:07.729534: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-17 14:52:07.729622: I tensorflow/compile

In [11]:
from keras.callbacks import EarlyStopping, ModelCheckpoint


es = EarlyStopping(monitor = 'accuracy', mode = 'min', verbose = 1, patience = 200)
mc = ModelCheckpoint('./model.h5', monitor = 'val_accuracy', mode = 'max', verbose = 1, save_best_only = True)

In [17]:
history_embedding = model.fit(X_train_pad, x_test, validation_data=(X_test_pad, y_test), epochs = 100, batch_size = 120, verbose = 1, callbacks= [es, mc]  )
# history_embedding = model.fit(X_train_pad,testdata, epochs = 35, batch_size = 120, validation_data=(X_test_pad, test_data.label),verbose = 1, callbacks= [es, mc]  )

Epoch 1/100


/home/klc/miniconda3/envs/tf174/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


8/8 [==============================] - ETA: 0s - loss: 1.0968 - accuracy: 0.4356
Epoch 1: val_accuracy improved from -inf to 0.55794, saving model to ./model.h5
8/8 [==============================] - 2s 172ms/step - loss: 1.0968 - accuracy: 0.4356 - val_loss: 1.0927 - val_accuracy: 0.5579
Epoch 2/100
8/8 [==============================] - ETA: 0s - loss: 1.0931 - accuracy: 0.5011
Epoch 2: val_accuracy did not improve from 0.55794
8/8 [==============================] - 1s 135ms/step - loss: 1.0931 - accuracy: 0.5011 - val_loss: 1.0875 - val_accuracy: 0.5579
Epoch 3/100
8/8 [==============================] - ETA: 0s - loss: 1.0894 - accuracy: 0.5011
Epoch 3: val_accuracy did not improve from 0.55794
8/8 [==============================] - 1s 143ms/step - loss: 1.0894 - accuracy: 0.5011 - val_loss: 1.0823 - val_accuracy: 0.5579
Epoch 4/100
8/8 [==============================] - ETA: 0s - loss: 1.0858 - accuracy: 0.5011
Epoch 4: val_accuracy did not improve from 0.55794
8/8 [===============

In [20]:

from keras.models import load_model  
new_model = load_model(r'./model.h5')
accuracy=new_model.evaluate(X_test_pad, y_test)[1]

8/8 [==============================] - 1s 91ms/step - loss: 1.0927 - accuracy: 0.5579


In [41]:
import scipy.stats
import scipy._lib._util
import scipy
from sklearn.metrics import confusion_matrix
import numpy as np
import scipy.stats
y_prediction = new_model.predict(X_test_pad, test_data.label,  232 // 120+1)

y_pred_bool = y_prediction>=0.5
trueClass = X_test_pad.classes[X_test_pad.index_array]
print(classification_report(y_true = trueClass, y_pred= y_pred_bool))
result = confusion_matrix(y_true = trueClass, y_pred= y_pred_bool)
print(result)

ImportError: cannot import name '_contains_nan' from 'scipy._lib._util' (/home/klc/miniconda3/envs/tf174/lib/python3.9/site-packages/scipy/_lib/_util.py)